In [6]:
from chatbot_modules import *
from datetime import datetime, timedelta
import concurrent.futures
import asyncio

In [7]:
today = datetime.today().strftime('%Y%m%d')

past_date = datetime.today() - timedelta(days=90)
past_date = past_date.strftime('%Y%m%d')

# 나중에 gpt로 종목명 -> 티커 변환하는거 해줘야됨
user_input_sample = '삼성전자에 대해 투자하는게 좋을까? 어떻게 생각해?'

In [8]:
async def get_raw_info_for_chatbot(past_date, today, ticker):
    loop = asyncio.get_event_loop()

    # 동기 함수 실행 (스레드 풀 사용)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_sp = executor.submit(get_stock_prices, past_date, today, ticker)
        future_dart = executor.submit(get_dart, ticker, past_date)
        future_fin_info = executor.submit(get_fin_info_from_naver, ticker)

        # 비동기 함수 실행 (asyncio.create_task 사용)
        future_news = asyncio.create_task(get_ticker_news(ticker))

        # 결과 수집
        sp = future_sp.result()
        print('주가데이터 수집 완료')
        fin_report, fin_statement = future_dart.result()
        print('재무보고서 및 재무제표 수집 완료')
        fin_info = future_fin_info.result()
        print('재무지표 수집 완료')
        ticker_news = await future_news  # 비동기 함수 결과 기다리기
        print('뉴스 정보 수집 완료')

    return sp, fin_report, fin_statement, ticker_news, fin_info

In [10]:
# 실행 예시
ticker = get_stock_code_fuzzy(user_input_sample)

if ticker:
    result = await get_raw_info_for_chatbot(past_date, today, ticker)
    print(result)
else:
    print('종목명 추출 실패')

주가데이터 수집 완료
재무보고서 및 재무제표 수집 완료
재무지표 수집 완료
뉴스 정보 수집 완료
(               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2024-11-11  56700  56800  55000  55000  29811326 -3.508772
2024-11-12  54600  54600  53000  53000  37962881 -3.636364
2024-11-13  52000  53000  50500  50600  52527995 -4.528302
2024-11-14  50200  51800  49900  49900  48510715 -1.383399
2024-11-15  50300  54200  50300  53500  46774485  7.214429
2024-11-18  57000  57500  55900  56700  48095234  5.981308
2024-11-19  56500  57500  55900  56300  31539632 -0.705467
2024-11-20  56100  56500  54800  55300  20864667 -1.776199
2024-11-21  54900  56900  54700  56400  19096850  1.989150
2024-11-22  56000  56700  55900  56000  15281543 -0.709220
2024-11-25  57400  57900  56700  57900  36237325  3.392857
2024-11-26  57900  58900  57500  58300  23209405  0.690846
2024-11-27  57700  57800  56000  56300  21808387 -3.430532
2024-11-28  56000  56400  55200  55500  20001134 -1.420959
2